### Slicing
- Slice the row labels 'Perry' to 'Potter' : `p_counties`
- Slice the row labels 'Perry' to 'Potter' in reverse order

In [1]:
import pandas as pd
election = pd.read_csv('C:\\Users\\saifs\\Desktop\\data_science\\datasets\\pennsylvania2012_turnout.csv', index_col = 'county')
election.head()

,state,total,Obama,Romney,winner,voters,turnout,margin
county,,,,,,,,
Adams,PA,41973,35.482334,63.112001,Romney,61156,68.632677,27.629667
Allegheny,PA,614671,56.640219,42.185820,Obama,924351,66.497575,14.454399
Armstrong,PA,28322,30.696985,67.901278,Romney,42147,67.198140,37.204293
Beaver,PA,80015,46.032619,52.637630,Romney,115157,69.483401,6.605012
Bedford,PA,21444,22.057452,76.986570,Romney,32189,66.619031,54.929118


In [2]:
p_counties = election.loc['Perry':'Potter']
p_counties

,state,total,Obama,Romney,winner,voters,turnout,margin
county,,,,,,,,
Perry,PA,18240,29.769737,68.591009,Romney,27245,66.948064,38.821272
Philadelphia,PA,653598,85.224251,14.051451,Obama,1099197,59.461407,71.172800
Pike,PA,23164,43.904334,54.882576,Romney,41840,55.363289,10.978242
Potter,PA,7205,26.259542,72.158223,Romney,10913,66.022175,45.898681


In [3]:
p_counties_reverse = election.loc['Potter':'Perry':-1]
p_counties_reverse

,state,total,Obama,Romney,winner,voters,turnout,margin
county,,,,,,,,
Potter,PA,7205,26.259542,72.158223,Romney,10913,66.022175,45.898681
Pike,PA,23164,43.904334,54.882576,Romney,41840,55.363289,10.978242
Philadelphia,PA,653598,85.224251,14.051451,Obama,1099197,59.461407,71.172800
Perry,PA,18240,29.769737,68.591009,Romney,27245,66.948064,38.821272


- Slice the columns from the starting column to 'Obama' and assign the result to left_columns
- Slice the columns from 'Obama' to 'winner' and assign the result to middle_columns
- Slice the columns from 'Romney' to the end and assign the result to right_columns

In [4]:
left_columns = election.loc[:,:'Obama']
middle_columns = election.loc[:,'Obama':'winner']
right_columns = election.loc[:,'Romney':]
display(left_columns.head(3))
display(middle_columns.head(3))
display(right_columns.head(3))

,state,total,Obama
county,,,
Adams,PA,41973,35.482334
Allegheny,PA,614671,56.640219
Armstrong,PA,28322,30.696985


,Obama,Romney,winner
county,,,
Adams,35.482334,63.112001,Romney
Allegheny,56.640219,42.185820,Obama
Armstrong,30.696985,67.901278,Romney


,Romney,winner,voters,turnout,margin
county,,,,,
Adams,63.112001,Romney,61156,68.632677,27.629667
Allegheny,42.185820,Obama,924351,66.497575,14.454399
Armstrong,67.901278,Romney,42147,67.198140,37.204293


### Setting NaN to specific values
- Filter the rows where the margin was less than 1. Then convert these rows of the 'winner' column to np.nan to indicate that these results are too close to declare a winner.

In [5]:
import numpy as np
election[election['margin'] < 1]['winner'] = np.nan

F:\data_science\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
election[election['margin'] < 1]['winner'] # the value is not set

county
Berks      Romney
Centre     Romney
Chester    Romney
Name: winner, dtype: object

### Returning a view versus a copy
- When setting values in a pandas object, care must be taken to avoid what is called chained indexing. Here is an example.

In [7]:
dfmi = pd.DataFrame([list('abcd'),list('efgh'),list('ijkl'),list('mnop')],
                    columns=pd.MultiIndex.from_product([['one', 'two'],['first', 'second']]))
dfmi

one          two       
  first second first second
0     a      b     c      d
1     e      f     g      h
2     i      j     k      l
3     m      n     o      p

In [8]:
dfmi['one']['second']

0    b
1    f
2    j
3    n
Name: second, dtype: object

In [9]:
dfmi.loc[:, ('one','second')]

0    b
1    f
2    j
3    n
Name: (one, second), dtype: object

These both yield the same results, so which should you use? It is instructive to understand the order of operations on these and why method 2 (`.loc`) is much preferred over method 1 (`chained []`).

`dfmi['one']` selects the first level of the columns and returns a DataFrame that is singly-indexed. Then another Python operation `dfmi_with_one['second']` selects the series indexed by `'second'`. This is indicated by the variable `dfmi_with_one` because pandas sees these operations as separate events. e.g. separate calls to `__getitem__`, so it has to treat them as linear operations, they happen one after another.

Contrast this to `df.loc[:,('one','second')]` which passes a nested tuple of `(slice(None),('one','second'))` to a single call to `__getitem__`. This allows pandas to deal with this as a single entity. Furthermore this order of operations can be significantly faster, and allows one to index both axes if so desired.

### Why does assignment fail when using chained indexing?
It turns out that assigning to the product of chained indexing has inherently unpredictable results. To see this, think about how the Python interpreter executes this code:
```Python
dfmi.loc[:, ('one', 'second')] = value
# becomes
dfmi.loc.__setitem__((slice(None), ('one', 'second')), value)
```
But this code is handled differently:
```Python
dfmi['one']['second'] = value
# becomes
dfmi.__getitem__('one').__setitem__('second', value)
```
See that `__getitem__` in there? Outside of simple cases, it’s very hard to predict whether it will return a view or a copy (it depends on the memory layout of the array, about which pandas makes no guarantees), and therefore whether the `__setitem__` will modify dfmi or a temporary object that gets thrown out immediately afterward. That’s what `SettingWithCopy` is warning you about!

### Back to setting NaN to specific values
- Filter the rows where the margin was less than 1. Then convert these rows of the 'winner' column to np.nan to indicate that these results are too close to declare a winner.

In [10]:
election.loc[election['margin'] < 1 , 'winner'] = np.nan

In [11]:
election.loc[election['margin'] < 1]

,state,total,Obama,Romney,winner,voters,turnout,margin
county,,,,,,,,
Berks,PA,163253,48.939376,49.528646,NaN,250356,65.208343,0.589269
Centre,PA,68801,48.948416,48.977486,NaN,112949,60.913333,0.029069
Chester,PA,248295,49.228539,49.650617,NaN,337822,73.498766,0.422079


### Filtering using NaNs
- Select the 'age' and 'cabin' columns of titanic and create a new DataFrame df
- Print the shape of df
- Remove rows where any of these two columns contains missing data and print the shape
- Remove rows where all of these two columns contain missing data and print the shape
- Drop columns from the titanic DataFrame that have less than 1000 non-missing values

In [12]:
titanic = pd.read_csv('C:\\Users\\saifs\\Desktop\\data_science\\datasets\\titanic.csv')
titanic.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [13]:
df = titanic[['age','cabin']]
print(df.shape)
print(df.dropna(how = 'any').shape)
print(df.dropna(how = 'all').shape)

(1309, 2)
(272, 2)
(1069, 2)


In [14]:
print(titanic.shape)
print(titanic.dropna(thresh = 1000, axis = 'columns').shape)

(1309, 14)
(1309, 10)


#### Using `.apply()` to transform a column

The `.apply()` method can be used on a pandas DataFrame to apply an arbitrary Python function to every element.
#### Using `.map()` with a dictionary

The `.map()` method is used to transform values according to a Python dictionary look-up.
- Use a dictionary to map the values 'Obama' and 'Romney' in the 'winner' column to the values 'blue' and 'red', and assign the output to the new column 'color'

In [15]:
red_vs_blue = {'Obama' : 'blue', 'Romney' : 'red'}
election['color'] = election['winner'].map(red_vs_blue)
election.head()

,state,total,Obama,Romney,winner,voters,turnout,margin,color
county,,,,,,,,,
Adams,PA,41973,35.482334,63.112001,Romney,61156,68.632677,27.629667,red
Allegheny,PA,614671,56.640219,42.185820,Obama,924351,66.497575,14.454399,blue
Armstrong,PA,28322,30.696985,67.901278,Romney,42147,67.198140,37.204293,red
Beaver,PA,80015,46.032619,52.637630,Romney,115157,69.483401,6.605012,red
Bedford,PA,21444,22.057452,76.986570,Romney,32189,66.619031,54.929118,red


**In order to manipulate index of DataFrame using custom function, use `.map()`**

In [16]:
election.index = election.index.str.upper()
election.index

Index(['ADAMS', 'ALLEGHENY', 'ARMSTRONG', 'BEAVER', 'BEDFORD', 'BERKS',
       'BLAIR', 'BRADFORD', 'BUCKS', 'BUTLER', 'CAMBRIA', 'CAMERON', 'CARBON',
       'CENTRE', 'CHESTER', 'CLARION', 'CLEARFIELD', 'CLINTON', 'COLUMBIA',
       'CRAWFORD', 'CUMBERLAND', 'DAUPHIN', 'DELAWARE', 'ELK', 'ERIE',
       'FAYETTE', 'FOREST', 'FRANKLIN', 'FULTON', 'GREENE', 'HUNTINGDON',
       'INDIANA', 'JEFFERSON', 'JUNIATA', 'LACKAWANNA', 'LANCASTER',
       'LAWRENCE', 'LEBANON', 'LEHIGH', 'LUZERNE', 'LYCOMING', 'MCKEAN',
       'MERCER', 'MIFFLIN', 'MONROE', 'MONTGOMERY', 'MONTOUR', 'NORTHAMPTON',
       'NORTHUMBERLAND', 'PERRY', 'PHILADELPHIA', 'PIKE', 'POTTER',
       'SCHUYLKILL', 'SNYDER', 'SOMERSET', 'SULLIVAN', 'SUSQUEHANNA', 'TIOGA',
       'UNION', 'VENANGO', 'WARREN', 'WASHINGTON', 'WAYNE', 'WESTMORELAND',
       'WYOMING', 'YORK'],
      dtype='object', name='county')

In [20]:
election.index = election.index.map(str.capitalize)

In [21]:
election.index

Index(['Adams', 'Allegheny', 'Armstrong', 'Beaver', 'Bedford', 'Berks',
       'Blair', 'Bradford', 'Bucks', 'Butler', 'Cambria', 'Cameron', 'Carbon',
       'Centre', 'Chester', 'Clarion', 'Clearfield', 'Clinton', 'Columbia',
       'Crawford', 'Cumberland', 'Dauphin', 'Delaware', 'Elk', 'Erie',
       'Fayette', 'Forest', 'Franklin', 'Fulton', 'Greene', 'Huntingdon',
       'Indiana', 'Jefferson', 'Juniata', 'Lackawanna', 'Lancaster',
       'Lawrence', 'Lebanon', 'Lehigh', 'Luzerne', 'Lycoming', 'Mckean',
       'Mercer', 'Mifflin', 'Monroe', 'Montgomery', 'Montour', 'Northampton',
       'Northumberland', 'Perry', 'Philadelphia', 'Pike', 'Potter',
       'Schuylkill', 'Snyder', 'Somerset', 'Sullivan', 'Susquehanna', 'Tioga',
       'Union', 'Venango', 'Warren', 'Washington', 'Wayne', 'Westmoreland',
       'Wyoming', 'York'],
      dtype='object', name='county')

### Using vectorized functions
When performance is paramount, you should avoid using `.apply(`) and `.map()` because those constructs perform Python for-loops over the data stored in a pandas Series or DataFrame. By using vectorized functions instead, you can loop over the data at the same speed as compiled code (C, Fortran, etc.)! NumPy, SciPy and pandas come with a variety of vectorized functions (called Universal Functions or UFuncs in NumPy).

- Import the zscore function from scipy.stats and use it to compute the deviation in voter turnout in Pennsylvania from the mean in fractions of the standard deviation.

In [22]:
from scipy.stats import zscore

In [27]:
turnout_zscore = zscore(election['turnout'])
election['turnout_zscore'] = turnout_zscore
election.head()

,state,total,Obama,Romney,winner,voters,turnout,margin,color,turnout_zscore
county,,,,,,,,,,
Adams,PA,41973,35.482334,63.112001,Romney,61156,68.632677,27.629667,red,0.853734
Allegheny,PA,614671,56.640219,42.185820,Obama,924351,66.497575,14.454399,blue,0.439846
Armstrong,PA,28322,30.696985,67.901278,Romney,42147,67.198140,37.204293,red,0.575650
Beaver,PA,80015,46.032619,52.637630,Romney,115157,69.483401,6.605012,red,1.018647
Bedford,PA,21444,22.057452,76.986570,Romney,32189,66.619031,54.929118,red,0.463391


### Changing index of a DataFrame
Indexes are immutable objects. This means that if you want to change or modify the index in a DataFrame, then you need to change the whole index.

### Multi-Indexing and Fancy Slicing

In [36]:
sales = pd.read_csv('C:\\Users\\saifs\\Desktop\\data_science\\datasets\\grocery_sales.csv')
sales

,month,eggs,salt,spam,state
0,1,47,12.0,17,CA
1,2,110,50.0,31,CA
2,1,221,89.0,72,NY
3,2,77,87.0,20,NY
4,1,132,NaN,52,TX
5,2,205,60.0,55,TX


In [37]:
sales = sales.set_index(['state','month'])
sales

eggs  salt  spam
state month                  
CA    1        47  12.0    17
      2       110  50.0    31
NY    1       221  89.0    72
      2        77  87.0    20
TX    1       132   NaN    52
      2       205  60.0    55

In [38]:
sales.sort_index()

eggs  salt  spam
state month                  
CA    1        47  12.0    17
      2       110  50.0    31
NY    1       221  89.0    72
      2        77  87.0    20
TX    1       132   NaN    52
      2       205  60.0    55

Looking up data based on inner levels of a MultiIndex can be a bit trickier. The trickiest of all these lookups are when you want to access some inner levels of the index. In this case, you need to use `slice(None)` in the slicing parameter for the outermost dimension(s) instead of the usual `:`, or use `pd.IndexSlice`

In [40]:
NY_month1 = sales.loc[('NY',1),:]
NY_month1

eggs    221.0
salt     89.0
spam     72.0
Name: (NY, 1), dtype: float64

In [42]:
CA_TX_month2 = sales.loc[(['CA','TX'],2), :]
CA_TX_month2

,,eggs,salt,spam
state,month,,,
CA,2,110,50.0,31
TX,2,205,60.0,55


In [44]:
all_month2 = sales.loc[(slice(None),2), :]
all_month2

,,eggs,salt,spam
state,month,,,
CA,2,110,50.0,31
NY,2,77,87.0,20
TX,2,205,60.0,55


## Pivoting DataFrames

Suppose you started a blog for a band, and you would like to log how many visitors you have had, and how many signed-up for your newsletter. To help design the tours later, you track where the visitors are. A DataFrame called users consisting of this information has been pre-loaded for you.
- Your job is to pivot users so that the focus is on 'visitors', with the columns indexed by 'city' and the rows indexed by 'weekday'
- Pivot the users DataFrame with both 'signups' and 'visitors' 

In [46]:
users = pd.reaad_csv('C:\\Users\\saifs\\Desktop\\data_science\\datasets\\users.csv', index_col = 0)
users

,weekday,city,visitors,signups
0,Sun,Austin,139,7
1,Sun,Dallas,237,12
2,Mon,Austin,326,3
3,Mon,Dallas,456,5


In [47]:
users.pivot(index = 'weekday', columns = 'city', values = 'visitors')

city,Austin,Dallas
weekday,,
Mon,326,456
Sun,139,237


In [48]:
users.pivot(index = 'weekday', columns = 'city')

visitors        signups       
city      Austin Dallas  Austin Dallas
weekday                               
Mon          326    456       3      5
Sun          139    237       7     12

### Unstacking
- Pivot a level of the (necessarily hierarchical) index labels.
- Returns a DataFrame having a new level of column labels whose inner-most level consists of the pivoted index labels.
- If the index is not a MultiIndex, the output will be a Series

In [ ]:
users = users.set_index(['city','weekday'])

In [55]:
users = users.sort_index()
users

visitors  signups
city   weekday                   
Austin Mon           326        3
       Sun           139        7
Dallas Mon           456        5
       Sun           237       12

`users` DataFrame is multi-indexed.
- Define a DataFrame `byweekday` with the 'weekday' level of users unstacked.

In [58]:
byweekday = users.unstack(level = 'weekday')
byweekday # inner level index move in inner column label

visitors      signups    
weekday      Mon  Sun     Mon Sun
city                             
Austin       326  139       3   7
Dallas       456  237       5  12

### Stacking
- Stack the prescribed level(s) from columns to index.
- Return a reshaped DataFrame or Series having a multi-level index with one or more new inner-most levels compared to the current DataFrame. The new inner-most levels are created by pivoting the columns of the current dataframe:
    - if the columns have a single level, the output is a Series;
    - if the columns have multiple levels, the new index level(s) is (are) taken from the prescribed level(s) and the output is a DataFrame.
- The new index levels are sorted.

`byweekday` have multi-level column
- Stack `byweekday` by 'weekday' and print it to check if you get the same layout as the original users DataFrame. 

In [60]:
users = byweekday.stack(level = 'weekday')
users

visitors  signups
city   weekday                   
Austin Mon           326        3
       Sun           139        7
Dallas Mon           456        5
       Sun           237       12

- Unstack users by 'city': bycity
- Stack 'bycity' by 'city' and print it

In [61]:
bycity = users.unstack(level = 'city')
bycity

visitors        signups       
city      Austin Dallas  Austin Dallas
weekday                               
Mon          326    456       3      5
Sun          139    237       7     12

In [65]:
newusers = bycity.stack(level = 'city')
newusers

visitors  signups
weekday city                     
Mon     Austin       326        3
        Dallas       456        5
Sun     Austin       139        7
        Dallas       237       12

In [66]:
users

visitors  signups
city   weekday                   
Austin Mon           326        3
       Sun           139        7
Dallas Mon           456        5
       Sun           237       12

### Restoring the index order
- Use `.swaplevel(0, 1)` to flip the index levels.
- Note they won't be sorted. To sort them, you will have to follow up with a `.sort_index()`

In [67]:
newusers = newusers.swaplevel(0, 1)
newusers

,,visitors,signups
city,weekday,,
Austin,Mon,326,3
Dallas,Mon,456,5
Austin,Sun,139,7
Dallas,Sun,237,12


In [70]:
newusers = newusers.sort_index()
newusers

visitors  signups
city   weekday                   
Austin Mon           326        3
       Sun           139        7
Dallas Mon           456        5
       Sun           237       12

In [69]:
print(users.equals(newusers))

True
